In [1]:
import torch
from torch.utils.data.dataset import Dataset
import torch.nn.functional as F
import torch.nn as nn
import torch.utils
import torch.distributions

from cnnlstmnetwork import *

In [2]:
s = 64
im1 = torch.rand(3,s,s)
im2 = torch.rand(3,s,s)
im3 = torch.rand(3,s,s)
#im4 = torch.rand(3,128,128)
#im5 = torch.rand(3,128,128)
out = torch.rand(2)

In [3]:
im_w = torch.stack([im1,im2,im3],0)
im_w = torch.unsqueeze(im_w,0)
im_w.shape

torch.Size([1, 3, 3, 64, 64])

In [5]:
model = CNNLSTMNet(ts=3)
print(model)
o = model(im_w)

CNNLSTMNet(
  (ts_cnn): TimeDistributed(
    (layers): ModuleList(
      (0): CNNEncoder(
        (cnnmodule): Sequential(
          (0): ConvBlock(
            (seq): Sequential(
              (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
              (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): ReLU()
            )
          )
          (1): ConvBlock(
            (seq): Sequential(
              (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
              (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): ReLU()
            )
          )
          (2): ConvBlock(
            (seq): Sequential(
              (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
              (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): ReLU()
            )
          )

In [7]:
def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

In [8]:
get_n_params(model)

14332929

In [145]:
ts = 5
model = TimeDistributed(CNNEncoder, ts, 3)
model1 = LSTMModule(1024, 128, 2, ts, 2)

In [146]:
o = model(im_w)

In [147]:
o.shape

torch.Size([1, 5, 1024])

In [148]:
o = model1(o)

In [149]:
o

tensor([[-0.0596, -0.0491]], grad_fn=<AddmmBackward>)

In [134]:
1024/4

256.0